In [ ]:

# face_capture.py
import cv2
import mediapipe as mp
import csv
import os
import numpy as np

# ---------------------------
# CONFIGURATION (Face Static)
# ---------------------------
FACE_LABEL    = "happy"
FACE_LM       = 468
FACE_FEAT_LEN = FACE_LM * 4                # 1872 values/frame
FACE_CSV      = "face_expressions.csv"

# ---------------------------
# HEADER CREATION
# ---------------------------
def write_header(path, lm_total):
    if os.path.exists(path) and os.stat(path).st_size > 0:
        return
    header = ["class"]
    for lm_idx in range(1, lm_total+1):
        header += [
            f"lm{lm_idx}_x",
            f"lm{lm_idx}_y",
            f"lm{lm_idx}_z",
            f"lm{lm_idx}_v",
        ]
    with open(path, "w", newline="") as f:
        csv.writer(f).writerow(header)
    print(f"Header written to {path}")

write_header(FACE_CSV, FACE_LM)

# ---------------------------
# LANDMARK EXTRACTION
# ---------------------------
def extract_landmarks(lm_list, expected_count):
    if not lm_list:
        return [0.0] * expected_count * 4
    vals = []
    for lm in lm_list.landmark:
        x = lm.x if lm.x==lm.x else 0.0
        y = lm.y if lm.y==lm.y else 0.0
        z = lm.z if lm.z==lm.z else 0.0
        v = lm.visibility if lm.visibility==lm.visibility else 0.0
        vals += [x, y, z, v]
    if len(vals) < expected_count*4:
        vals += [0.0] * (expected_count*4 - len(vals))
    return vals

# ---------------------------
# CAPTURE LOOP
# ---------------------------
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as holistic:
    print("Press 'q' to quit face capture.")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        frame = cv2.flip(frame,1)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = holistic.process(img_rgb)

        face_feats = extract_landmarks(res.face_landmarks, FACE_LM)
        row = [FACE_LABEL] + face_feats
        with open(FACE_CSV, "a", newline="") as f:
            csv.writer(f).writerow(row)

        cv2.putText(frame, "Capture Face", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        cv2.imshow("Face Capture", frame)
        if cv2.waitKey(1)&0xFF==ord('q'): break
cap.release()
cv2.destroyAllWindows()
